# ノートブック型 Spark サービスにおける DevOps の実践

## ノートブック型 Spark サービスにおける DevOps の概要

ノートブック型 Spark サービスにおける CI/CD（Continuous Integration（継続的インテグレーション）/ Continuous Delivery（継続的デリバリー））の実施方法を説明する。CI/CD とは、コード開発から本番環境への統合までの品質の保証と期間の最小化を担保しつつ、変更点の統合を行いながらシステム運用を行うプロセスである。

CI/CD は次のようなプロセスを実施することが多い。CI にてリリース可能なリソース（一般的には main ブランチのコード）の準備を行い、 CD にて他リソースと統合した本番相当の環境で検証を行ったうえで本番環境へリリースする。リソースの管理は Git で行い、 Git の運用ルールは Git-flow と呼ぶ。Git-flow はチームのスキルセットに応じて方針を定める必要があり、Git の機能を多様するような複雑な設計を行うと運用コストが高くなる可能性がある。継続的な開発と運用を行えるように、シンプルな設計が望ましい。CI/CD の処理群を CI/CD パイプラインと呼び、Github Actions や Azure Pipelines などのサービスで実行する。

- CI
  - Code
    - Git レポジトリ（ main ブランチ）を準備。
    - main ブランチから派生した開発用レポジトリー（フォークしたレポジトリ、あるいは、ブランチ）を準備。
    - プログラムコードとテストコードを開発。
    - 開発用レポジトリーへ変更内容をコミット。
    - Main ブランチへ変更要求（Pull Requet）を実施。
  - Build
    - main ブランチをベースとして PR の変更内容を反映したコードを生成。
    - ライブラリ等のビルド。
    - テスト用環境の準備。
    - テストを実行。
    - リリースの実施有無を判断。
  - Release
    - 環境へデプロイ可能なリソースを生成。
- CD
  - Deploy
    - 環境へリソースを配布。
  - Test
    - 環境にてテスト実行（主に他システムとの連携テストやE2Eテストを実施）。
  - Opeate
    - システムの運用。

Code や Build では、開発支援の観点、及び、コードの品質管理の観点により、次のような項目の実施が求められる。

1. Linter ツールによる静的解析
1. Formatter ツールによるコード整形
1. テスト結果の取得、及び、テスト結果の継続的な管理
1. テストのコードカバレッジの取得、及び、テストのコードカバレッジの継続的な管理

## Databricks での開発

### Databricks の開発方針

Databricks Repos のファイル管理仕様、Databricks Workspace 上ではノートブックであるが Git 上では Python ファイルとして管理される仕様、を考慮して、次のような開発指針とする。

- 共通機能を保持したノートブックを`%run`により呼び出して開発を行うこと
- テストケースを unittest で記載し、ノートブック上で正常終了することを確認すること
- ローカル環境での動作を考慮したコードを記載すること

ローカル環境での動作を行う目的は、ノートブック型環境で実施できないことを補うためである。ノートブック型環境には、Visual Studio Code (VS Code) などのローカル環境と比較すると、次のような特徴がある。ノートブック型環境の短所を克服するために、ノートブック型環境とローカル環境の併用がおすすめ。

- Pros
  1. 実行ノートブックにおけるコードと結果を合わせて保存できること
  1. 開発環境の準備が最小限となり、実行環境とコードの共有ができること
  1. 処理をセルごとに分割することで処理フローの解釈が容易となること
- Cons
  1. デバック実行が不可
  1. Linter（静的解析）ツールの利用不可
  1. Formatter （コード整形）ツールの利用不可
  1. テストのコードカバレッジの取得不可

### ローカル環境（Visual Studio Code） 上での開発

#### ローカル環境での開発方針

ローカル環境（Visual Studio Code）の開発環境の構築方法として下記表の方法が考えられるが、ノートブック型環境の短所を補うための補足的な利用であるため、1 の方法を採用。CI/CD パイプラインの実行時 Databricks 上で実施するため、Databricks に近い環境を選択。

| #    | 方法                                      | Pros                                                         | Cons                                                         |
| ---- | ----------------------------------------- | ------------------------------------------------------------ | ------------------------------------------------------------ |
| 1    | Databricks Connect により開発する方法     | - VS Code 上でテスト実施でき、コードカバレッジも取得可能     | - 一部機能を利用することができず、新機能の開発の計画がない                           |
| 2    | dbx by Databricks Labs により開発する方法 | - Databricks Workspace ローカル上のコードを、ローカル環境と同期が可能 | - ベンダーのサポートがない<br />- コードカバレッジを取得する方法を確立できていない |
| 3    | ローカル環境の Spark 環境にて開発する方法 | - VS Code 上でテスト実施でき、コードカバレッジも取得可能     | - Spark プロバイダー固有機能のテストを実施できない           |

Databricks Connect 環境の構築方法としては下記表の方法が考えられるが、１の方法を採用。ローカル環境の OS が Windows の場合には、Databricks で利用されている OS との仕様差異を最小限とするために、Databricks Runtime と同じバージョンの OS (Databricks Runtime 10.4 LTS を利用する場合には Ubuntu 20.04 LTS) を用いることを推奨。

| #    | 方法                                                         | Pros                                             | Cons                                                         |
| ---- | ------------------------------------------------------------ | ------------------------------------------------ | ------------------------------------------------------------ |
| 1    | Conda （無償：Miniconda、有償：Anaconda）を用いて構築する方法 | - 初期環境の統一ができ、環境を柔軟に切り替え可能 | - 環境の統一が困難                                           |
| 2    | Poetry を用いて構築する方法                                  | - 環境の統一が容易                               | - Databricks Connect のバージョンを切り替えることができない |
| 3    | コンテナ（無償：Rancher Desktop、有償：Docker Desktop）を用いて構築するする方法             | - 初期環境の統一ができ、環境を柔軟に切り替え可能  | - コンテナーに関する知識が必要 |
| 4    | Python 仮想環境を用いずに直接構築する方法             | - 特になし                                       | - Databricks Connect のバージョンを切り替えることができない |

Visual Studio Code の設定値を開発メンバーで共有するために、次のファイルを`.vscode`ディレクトリに配置。

| #    | ファイル名      | 概要                                                      |
| ---- | --------------- | --------------------------------------------------------- |
| 1    | settings.json   | Visual Studio Code の設定を保持。                         |
| 2    | extensions.json | Visual Studio Code に追加推奨の拡張機能のリストを保持。   |
| 3    | launch.json     | Visual Studio Code にてデバッグ実行する際のの設定を保持。 |

Python の Linter と Formatter として次のライブラリを利用。

| #    | 分類      | ライブラリ名 | 目的                               |
| ---- | --------- | ------------ | ---------------------------------- |
| 1    | Formatter | black        | コードに対するフォーマットツール。             |
| 2    | Formatter | isort        | import文に対するフォーマットツール。           |
| 3    | Linter    | flake8       | コードに対する静的解析ツール。     |
| 4    | Linter    | pydocstyle   | Docstring に対する静的解析ツール。 |
| 5    | Linter    | mypy         | 型に対する静的解析ツール。         |

ライブラリの設定値は、次のファイルにて設定。

| #    | ファイル名     | 概要                                                         |
| ---- | -------------- | ------------------------------------------------------------ |
| 1    | pyproject.toml | 基本的な設定値を設定。                                       |
| 2    | .vscode/settings.json  | ローカル環境でのみ有効な設定値を設定。テスト実行時の並列数の設定など。 |
| 3    | .vscode/launch.json    | デバッグ時にのみ有効な設定値を設定。カバレッジレポートを取得しない設定など。 |


Conda 環境情報を`environment.yml`に記述しており、Databricks Connect に対応した Python バージョンをベースに`requirements.txt`を pip install する構成。`requirements.txt`では、Databricks Connect や Linter などの開発時に必要なライブラリだけでなく、Databricks Runtime にインストールされている多くのライブラリを保持。

```requirements.txt
# Formatting libraries
black
isort

# Linter libraries
flake8
pydocstyle
mypy

# test libraries
pytest
coverage
pytest-cov
pytest-xdist

# Sphinx Libraries
sphinx
sphinx-markdown-builder

# databricks Connect
databricks-connect==10.4.*
```

Databricks Runtime インストールされているライブラリのバージョンを取得するために、次のような手順で `requirements.txt` を作成している。Conda 環境構築時にエラーとなったライブラリーを除外。

1. `%pip freeze`をノートブック上で実行することで、クラスターにインストールされているライブラリ一覧を取得
2. `requirements.txt`に、１の結果を貼り付け、不要なライブラリを削除
3. `conda env create -f environment.yml`により環境が構築できることを確認
4. `.environments` ディレクトリ下に Databricks Connect のバージョンごとの環境情報を保存

In [0]:
%pip freeze

appdirs==1.4.4
argon2-cffi==20.1.0
async-generator==1.10
attrs==20.3.0
backcall==0.2.0
bidict==0.21.4
bleach==3.3.0
boto3==1.16.7
botocore==1.19.7
certifi==2020.12.5
cffi==1.14.5
chardet==4.0.0
coverage==6.5.0
cycler==0.10.0
Cython==0.29.23
dbus-python==1.2.16
decorator==5.0.6
defusedxml==0.7.1
distlib==0.3.4
distro==1.4.0
distro-info===0.23ubuntu1
entrypoints==0.3
facets-overview==1.0.0
filelock==3.6.0
idna==2.10
iniconfig==1.1.1
ipykernel==5.3.4
ipython==7.22.0
ipython-genutils==0.2.0
ipywidgets==7.6.3
jedi==0.17.2
Jinja2==2.11.3
jmespath==0.10.0
joblib==1.0.1
jsonschema==3.2.0
jupyter-client==6.1.12
jupyter-core==4.7.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.0
kiwisolver==1.3.1
koalas==1.8.2
MarkupSafe==2.0.1
matplotlib==3.4.2
mistune==0.8.4
nbclient==0.5.3
nbconvert==6.0.7
nbformat==5.1.3
nest-asyncio==1.5.1
notebook==6.3.0
numpy==1.20.1
packaging==20.9
pandas==1.2.4
pandocfilters==1.4.3
parso==0.7.0
patsy==0.5.1
pexpect==4.8.0
pickleshare==0.7.5
Pillow==8.2.0
plotly==5.5.0
pluggy==1.0.0
prometheus-client==0.10.1
prompt-toolkit==3.0.17
protobuf==3.17.2
psycopg2==2.8.5
ptyprocess==0.7.0
py==1.11.0
pyarrow==4.0.0
pycparser==2.20
Pygments==2.8.1
PyGObject==3.36.0
pyparsing==2.4.7
pyrsistent==0.17.3
pytest==7.1.3
pytest-cov==4.0.0
python-apt==2.0.0+ubuntu0.20.4.8
python-dateutil==2.8.1
python-engineio==4.3.0
python-socketio==5.4.1
pytz==2020.5
pyzmq==20.0.0
requests==2.25.1
requests-unixsocket==0.2.0
s3transfer==0.3.7
scikit-learn==0.24.1
scipy==1.6.2
seaborn==0.11.1
Send2Trash==1.5.0
six==1.15.0
ssh-import-id==5.10
statsmodels==0.12.2
tenacity==8.0.1
terminado==0.9.4
testpath==0.4.4
threadpoolctl==2.1.0
tomli==2.0.1
tornado==6.1
traitlets==5.0.5
unattended-upgrades==0.1
urllib3==1.25.11
virtualenv==20.4.1
wcwidth==0.2.5
webencodings==0.5.1
widgetsnbextension==3.5.1

#### ローカル環境の構築

ローカル環境として、Windows Subsystem for Linux (WSL)上に環境を構築する手順を実施。

1. WSL 上の Ubuntu 20.04 環境をエクスポートして別名でインポート
1. miniconda をインストール
1. conda env を作成
1. Databricks  への接続設定

**0.事前準備**

事前準備として、以下のインストールを事前に実施。

- [Windows Terminal](https://apps.microsoft.com/store/detail/9N0DX20HK701?hl=ja-jp&gl=JP)
- [Visual Studio Code](https://code.visualstudio.com/download)
  - [Remote Development](https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.vscode-remote-extensionpack)
- [Windows Subsystem for Linux](https://learn.microsoft.com/ja-jp/windows/wsl/install)
  - [Ubuntu 20.04 - Microsoft Store アプリ](https://apps.microsoft.com/store/detail/ubuntu-2004/9N6SVWS3RX71?hl=ja-jp&gl=jp)

**1. WSL 上の Ubuntu 20.04 環境をエクスポートして別名でインポート**

Windows Terminal にて powershell を起動後、以下のコマンドを実行

```powershell
# Set wsl distro name
$src_wsl_distro_name = "Ubuntu-20.04"
$tgt_wsl_distro_name = "databricks-connect"
```

```powershell
# Update wsl version to 2
wsl --set-version $src_wsl_distro_name 2
```

```powershell
$wsl_file_name = $src_wsl_distro_name + ".tar"
wsl --export $src_wsl_distro_name $wsl_file_name
```


```powershell
# Import wsl distro_name
wsl --import $tgt_wsl_distro_name $tgt_wsl_distro_name $wsl_file_name
```

```powershell
# Check wsl dstro list
wsl -l

# login to wsl distro
wsl -d $tgt_wsl_distro_name -u root
```

`default_user_name` 変数にて、ubuntu で作成したユーザー名をセットして、以下のコマンドを実行。

```bash
# Set default user name
default_user_name=wsl_user

cat << EOF > /etc/wsl.conf
[user]
default=$default_user_name
EOF

# chekc wsl.conf
cat /etc/wsl.conf
```

```bash
# logout
exit
```

```powershell
# Delete distro file
Remove-Item $wsl_file_name
```

```powershell
# reboot and re-login
wsl -t $tgt_wsl_distro_name
wsl -d $tgt_wsl_distro_name
```

```bash
sudo apt update && sudo apt upgrade -y
```

```bash
# install java required by databricks-connect
sudo apt install openjdk-8-jre -y
```

```bash
# check java version
java -version
```

**2. miniconda をインストール**

Windows Terminal を再起動後、 1 の手順で作成した環境のコマンドラインシェルを起動後、次のコマンドを実行。

```bash
# download miniconda
cd
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
```

```bash
# install miniconda
bash Miniconda3-latest-Linux-x86_64.sh
```

```bash
# delete miniconda sh file
rm Miniconda3-latest-Linux-x86_64.sh
```

```bash
# create directory for git
mkdir source
```

**3. conda env を作成**

VS Code から、1 の手順で作成した WSL 環境に接続後、次の手順により、Git Provider(Azure Repos)からコードをクローン。

1. Azure DevOps にてクローンする レポジトリーの `Repos` -> `Files`ページに接続後、`clone`を選択
1. `IDE`にある`Clone in VS Code`を選択し、VS Code に画面に切り替わることを確認
1. `Choose a folder to clone` プロンプトにて、 2 の手順で作成した `source` を選択
1. Azure DevOps に戻り、`Generate Git Credentials`を選択し、`Password` の値を取得
1. VS Code にて、`Password` を入力後、コードがクローンされることを確認
1. クローンしたディレクトリを開き、Azure DevOps 上のディレクトリ構成となっていることを確認
1. `Do you want to install the recommended extensions for this repository` プロンプトが表示された場合には、`install` を選択

クローン完了後、VS Code のターミナルにて、次のコマンドを実行。`databricks-connect` のファイルサイズが大きいことでタイムアウトが発生する場合には、`pip istall` により`databricks-connect`を個別にインストールし、`pip install -r requirements.txt`によりその他のライブラリをインストールする手順に変更する。

```bash
# create conda envrioment for databricks connect
conda env create -f environment.yml
```

```bash
# check conda env
conda env list
```

**4. Databricks  への接続設定**

VS Code のターミナルにて、次のコマンドを実行。設定値については、[ドキュメント](https://learn.microsoft.com/ja-jp/azure/databricks/dev-tools/databricks-connect#step-2-configure-connection-properties)を参照。通常の手順では、`.databricks-connect`にて接続情報が格納される。

```bash
# connect to databricks connect conda env
conda activate dbconnect_10_4

# config databricks connect
databricks-connect configure
```

```bash
# test connect to databricks cluster
databricks-connect test
```
Databricks Connect の接続確認後、次の手順を実施。

1. VS Code にて、`Ctrl+Shift+P` -> `Python:Select Interpreter` を選択後、作成した Conda 環境（`dbconnect_10_4`）を選択。
1. タスクバーにある VS Code アイコンを右クリックし、現在開いているディレクトリ（例：`spark_engineering_template[wsl: databrikcs-connect]``）の`一覧にピン留めする` を選択

## Azure DevOps (Pipelines) による CI/CD パイプライン

### Azure Pipelines 概要

Azure Pipelines とは、Azure DevOps の１機能であり、 CI/CD（Continuous Integration（継続的インテグレーション）/ Continuous Delivery（継続的デリバリー））を自動化させる pipeline を構築できるサービスである。pipeline は、stage -> job -> task の階層構造となってり、Trigger により実行され、 Stage により論理的境界線を定義し、 job にエージェントを紐づけて task で指定した処理を行う。

-  [Azure Pipelines New User Guide - Key concepts - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/get-started/key-pipelines-concepts?view=azure-devops)

Azure Pipelines では、エージェントの並列実行数が主な課金対象であり、複数人での開発を行う際には無償で並列数に制限なく利用できるエージェントレスジョブを利用するなどのエージェントの利用方法が重要となる。処理が完了するまでの待機処理を、通常のジョブで実行した場合にはエージェントを占有してしまうが、エージェントレスジョブで実行することで他パイプライン（他ジョブ）がエージェントを利用することができる。

- [Azure Pipelines エージェント - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/agents/agents?view=azure-devops&tabs=browser)
- [エージェントレス ジョブでサポートされているタスク - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/process/phases?view=azure-devops-2020&tabs=yaml#agentless-tasks)

Azure Pipelines では、YAML 形式で記述を行う。YAMLで記述できる項目は、Azure DevOps のドキュメントにて整理されており、タスク等の拡張機能をインストールすることも可能である。[Command Line task](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/tasks/utility/command-line?view=azure-devops&tabs=yaml) では`script`と記述できるなど、一部のタスクではシンタックスを利用することもできる。以前から Azure DevOps （Visual Studio Team Services） で利用できたクラシックパイプラインという開発方法もあるが、本記事では利用対象としない。クラシックパイプラインの機能をベースに解説されている記事もあるため、どちらの開発方法の記事であるかを確認する必要がある。

-   [YAML schema reference | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/yaml-schema/?view=azure-pipelines&viewFallbackFrom=azure-devops)
-   [Catalog of the built-in tasks for build-release and Azure Pipelines & TFS - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/tasks/?view=azure-devops)
-   [Azure Pipelines Extensions - Azure DevOps Services (visualstudio.com)](https://marketplace.visualstudio.com/search?target=AzureDevOps&category=Azure Pipelines&sortBy=Installs)

stages 、 jobs 、 tasks 、variables を別の YAML ファイルで定義し、テンプレートとして呼び出すことも可能。

- [Templates - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/process/templates?view=azure-devops)
  -   [stages.template definition | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/yaml-schema/stages-template?view=azure-pipelines)
  -   [jobs.template definition | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/yaml-schema/jobs-template?view=azure-pipelines)
  -   [steps.template definition | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/yaml-schema/steps-template?view=azure-pipelines)
  -   [variables.template definition | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/yaml-schema/variables-template?view=azure-pipelines)


機微な情報をパイプラインで利用する際には、シークレット変数とする方法、および、Azure Key Vault を利用する方法が推奨されており、YAML 上に直接記述するべきではない。

-   [シークレット変数を設定する - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/process/set-secret-variables?view=azure-devops&tabs=yaml%2Cbash)
-   [Azure Pipelines で Azure Key Vault シークレットを使用する - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/release/azure-key-vault?view=azure-devops&tabs=yaml)


パイプライン実行の承認（Approvals）や同時実行の制御（Exclusive lock）も実施できる。

-   [Approvals - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/en-us/azure/devops/pipelines/process/approvals?view=azure-devops&preserve-view=true&tabs=check-pass#approvals)
-   [Exclusive lock - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/en-us/azure/devops/pipelines/process/approvals?view=azure-devops&preserve-view=true&tabs=check-pass#exclusive-lock)


ジョブが異なる場合には異なるエージェント環境となるため、変数の受け渡しを行う際には次の記事で記載されているような記述を行う。`dependsOn`により変数取得元のジョブ間に依存関係を明示する必要がある。

- [Support for output variables - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/en-us/azure/devops/pipelines/process/deployment-jobs?view=azure-devops#support-for-output-variables)


ビルド番号などの定義済み変数については、次のドキュメントに記載されている。

- [実行 (ビルド) 番号 - Azure Pipelines | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/devops/pipelines/process/run-number?view=azure-devops&tabs=yaml)

Azure pipelins の実装方法を検討する際には、次のドキュメントの方法を試すことが参考になる。

- [Python アプリのビルドとテスト - Azure Pipelines | Microsoft Docs](https://docs.microsoft.com/ja-jp/azure/devops/pipelines/ecosystems/python?view=azure-devops)
- [Azure DevOps を使用した Azure Databricks での継続的インテグレーションとデリバリー - Azure Databricks | Microsoft Docs](https://docs.microsoft.com/ja-jp/azure/databricks/dev-tools/ci-cd/ci-cd-azure-devops)

Python アプリのビルドとテストを実施する際には、次のようなシンプルなパイプラインを実行できる。

```yaml
trigger: none
pr: none

jobs:
- job:
  displayName: execute and publish test
  pool:
    vmImage: 'ubuntu-latest'
  steps:
  - task: UsePythonVersion@0
    inputs:
      versionSpec: '3.7'
  - script: python -m pip install --upgrade pip setuptools wheel
    displayName: 'Install tools'
  - script: pip install -r requirements.txt
    displayName: 'Install requirements'
  - script: |
      pip install pytest pytest-azurepipelines
      pip install pytest-cov
      pytest --doctest-modules --junitxml=junit/test-results.xml --cov=. --cov-report=xml
    displayName: 'pytest'
  - task: PublishTestResults@2
    condition: succeededOrFailed()
    inputs:
      testResultsFiles: '**/test-*.xml'
      testRunTitle: 'Publish test results for Python $(python.version)'
  - task: PublishCodeCoverageResults@1
    inputs:
      codeCoverageTool: Cobertura
      summaryFileLocation: '$(System.DefaultWorkingDirectory)/**/coverage.xml'
```

### Databricks における CI/CD パイプラインの構築

#### Azure Pipelines から Databricks を操作する方法

Azure Pipelines から Databricks を操作するには、次の方法を実施する。

1. CmdLineタスクにより Databricks REST API を実行する方法
1. CmdLineタスクにより Databricks CLI を実行する方法

```yaml
# CmdLineタスクにより Databricks REST API を実行する方法
jobs:
  - job: Databrics_REST_API_by_CmdLine
    steps:
    - task: CmdLine@2
      displayName: Execute Databricks Workflow
      inputs:
        script: |
          results=$(curl -X POST $(DATABRICKS_WORKSPACE_URL)/api/2.1/jobs/run-now \
          -H 'Content-Type: application/json' \
          -H 'Authorization: Bearer $(DATABRICKS_TOKEN)' \
          -d '{"job_id": "$(DATABRICKS_JOB_ID)"}' \
          )
          echo $results
```

```yaml
# CmdLineタスクにより Databricks CLI を実行する方法
jobs:
  - job: Databrics_CLI
    steps:
    - task: UsePythonVersion@0
      displayName: 'Use Python 3.8'
    - task: CmdLine@2
      inputs:
        script: |
          pip install databricks-cli -q
          databricks jobs run-now --job-id $(DATABRICKS_JOB_ID)
      env:
        DATABRICKS_HOST: $(DATABRICKS_WORKSPACE_URL)
        DATABRICKS_TOKEN: $(DATABRICKS_TOKEN)
```

API については、次のドキュメントが参考になる。

- [Databricks CLI - Azure Databricks | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/databricks/dev-tools/cli/)
- [REST API (latest) | Databricks on AWS](https://docs.databricks.com/dev-tools/api/latest/index.html)


CI / CD 時には、サービスプリンシパル、あるいは、システムアカウントを利用することが推奨。

>  Azure AD サービス プリンシパルとその Azure AD トークンを使用するか、ワークスペース ユーザー用の Azure Databricks 個人用アクセス トークンを使用して、CI/CD プラットフォームに Azure Databricks リソースへのアクセス権を付与することをお勧めします。

引用元：[CI/CD のサービス プリンシパル - Azure Databricks | Microsoft Learn](https://learn.microsoft.com/ja-jp/azure/databricks/dev-tools/ci-cd/ci-cd-sp)

#### Databricks における CI パイプラインの構築

CI パイプラインをとしては、次のような処理を実行している。`Python スクリプトでの単体テストの実施` にて、複数の Databricks ランタイムでテストを実施している。

1. ノートブックでの単体テストの実施
    - Databricks Repos を作成
    - Databricks Workflows(Notbook Type) でのテストの実施とテスト結果の発行
      - Databricks Workflows(Notbook Type) の作成と実行
      - Databricks Workflows 処理の完了確認
      - テスト結果（JUnit 形式の XML ファイル）を Azure Pipelines に発行
1. Python スクリプトでの単体テストの実施
    - コードを DBFS にコピー
    - Databricks Workflows(Python script type)でのテストの実施とテスト結果の発行
      - Databricks Workflows(Python script type)の作成と実行
      - Databricks Workflows 処理の完了確認
      - テスト結果（JUnit 形式の XML ファイル）を Azure Pipelines に発行
1. 単体テストのコードカバレッジの発行
    - コードカバレッジ結果の統合と発行
      - DBFS からコードカバレッジ結果（.coverage）をコピー
      - コードカバレッジ結果の統合（ combine ）
      - コードカバレッジ結果を Cobertura 形式の XML ファイルへ変換
      - コードカバレッジ結果を発行
1. 事後処理
    - DBFS 上のコード等を削除
    - Databricks Repos 上のディレクトリを削除


想定のトリガー

1. Pull Request 作成がされたとき
2. main レポジトリー、あるいは、release/* レポジトリーに対して Merge 処理がされたとき

#### Databricks における CD パイプラインの構築

CD パイプラインとしては、状況に合わせて統合テスト等を含めるなどの対応を行う。

Databricks Repos 、あるいは、 Databricks Workspace にコードを配置する際には、次の方法を実施。

1. Databricks Repos を用いる場合には、[Repos API](https://docs.databricks.com/dev-tools/api/latest/repos.html) を利用
1. Databricks Workspace を用いる場合には、[Databricks CLI](https://learn.microsoft.com/ja-jp/azure/databricks/dev-tools/cli/) を利用


次のトリガーを想定。誤ってデプロイが実施されないように、パイプライン実行の承認の設定が推奨。

1. Manual トリガー
2. スケジュール設定